<a href="https://www.kaggle.com/code/preunpatching/ollama-runtime-for-kaggle?scriptVersionId=226513458" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Ollama Runtime for Kaggle
This notebook allows you to quickly run Ollama on Kaggle's servers, which can be useful for most people who do not have a powerful enough PC to run certain models.
## How to run
### Start notebook
It's recommended that the runtime is configured to use 2x T4 GPUs. To do that, go to Settings > Accelerator and set it to "GPU T4 x2". The runtime may run with just a CPU, however this will cause serious performance degredations compared to with a GPU.
Once ready, press Ctrl+Shift+Alt+Enter, or go to Runtime > Run all.
### Start model
When the notebook asks you to select a compatible Ollama model to use, you may go to https://ollama.com/models to see the library of available models. **Be aware that not all models will run on the best free version of the runtime (29 GB RAM, 2x T4 GPU with 32 GB VRAM), so pick the correct model to use!**
## Bugs
*   None

Once done, you're ready to chat with your selected model!
Have fun!

In [1]:
# Check if we're using an Nvidia GPU.
import os
if os.path.isfile("/opt/bin/nvidia-smi"):
  print("Nvidia GPU detected.")
  # Install pciutils so that Ollama can auto-detect the Nvidia GPU.
  !sudo apt install pciutils
else:
  print("Couldn't detect Nvidia GPU. This will cause serious performance degredations when using a CPU only. It is recommended that you switch to a Nvidia GPU to prevent performance degredations.")
  input("Press ENTER to continue . . .")
# Install Ollama, as well as its API and BS4 for model search function.
!pip install ollama bs4
!curl -fsSL https://ollama.com/install.sh | sh
print("\033[0m", end='')
# Start Ollama server in a seperate process.
import subprocess
import signal
process = subprocess.Popen("ollama serve", shell=True)
# Wait for Ollama server to initialize.
import time
time.sleep(0.5)
# Prompt user for which Ollama model to use.
model = ""
while not model:
  model = input("Select compatible Ollama model to use: ")
# Pull selected model.
from tqdm import tqdm
from ollama import pull
current_digest, bars = '', {}
for progress in pull(model, stream=True):
  digest = progress.get('digest', '')
  if digest != current_digest and current_digest in bars:
    bars[current_digest].close()

  if not digest:
    print(progress.get('status'))
    continue

  if digest not in bars and (total := progress.get('total')):
    bars[digest] = tqdm(total=total, desc=f'pulling {digest[7:19]}', unit='B', unit_scale=True)

  if completed := progress.get('completed'):
    bars[digest].update(completed - bars[digest].n)

  current_digest = digest
# Start model.
from ollama import chat
messages = []
while True:
  try:
    user_input = input('>>> ')
    if user_input.lower() in ["/bye"]:
      print("Terminating Ollama server...")
      process.send_signal(signal.SIGTERM)
      try:
          process.wait(timeout=5)  # Wait up to 5 seconds
          print("Ollama server terminated gracefully.")
      except subprocess.TimeoutExpired:
          print("Terminating Ollama server forcefully...")
          process.send_signal(signal.SIGKILL) # Force termination
          process.wait()
          print("Ollama server terminated forcefully.")
      break
    elif user_input.lower() in ["/clear"]:
      messages = []
      print("Cleared session context")
    else:
      response = ''
      for part in chat(
        model,
        messages=messages
        + [
          {'role': 'user', 'content': user_input},
        ],
        stream=True,
      ):
        response = response + part['message']['content']
        print(part['message']['content'], end='', flush=True)

      # Add the response to the messages to maintain the history.
      messages += [
        {'role': 'user', 'content': user_input},
        {'role': 'assistant', 'content': response},
      ]
  except KeyboardInterrupt:
    print("Terminating Ollama server...")
    process.send_signal(signal.SIGTERM)
    try:
        process.wait(timeout=5)  # Wait up to 5 seconds
        print("Ollama server terminated gracefully.")
    except subprocess.TimeoutExpired:
        print("Terminating Ollama server forcefully...")
        process.send_signal(signal.SIGKILL) # Force termination
        process.wait()
        print("Ollama server terminated forcefully.")
    break

Nvidia GPU detected.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpci3 pci.ids
The following NEW packages will be installed:
  libpci3 pci.ids pciutils
0 upgraded, 3 newly installed, 0 to remove and 129 not upgraded.
Need to get 343 kB of archives.
After this operation, 1,581 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 pci.ids all 0.0~2022.01.22-1 [251 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpci3 amd64 1:3.7.0-6 [28.9 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 pciutils amd64 1:3.7.0-6 [63.6 kB]
Fetched 343 kB in 0s (1,342 kB/s)  
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline



Select compatible Ollama model to use:  qwq


pulling manifest


pulling c62ccde5630c: 100%|██████████| 19.9G/19.9G [01:06<00:00, 300MB/s]  
pulling 41190096a061: 100%|██████████| 1.23k/1.23k [00:01<00:00, 982B/s]  
pulling d18a5cc71b84: 100%|██████████| 11.3k/11.3k [00:01<00:00, 9.20kB/s]
pulling 4afe5edfdb51: 100%|██████████| 77.0/77.0 [00:01<00:00, 65.4B/s]
pulling 6a8faa2fb8b0: 100%|██████████| 488/488 [00:00<00:00, 495B/s]  


verifying sha256 digest
writing manifest
success


...

<think>

Okay, so I need to figure out whether the egg or the chicken came first. Hmm, this is a classic question that's been around for ages. Let me start by breaking it down.

First, if we think about evolution, chickens evolved from earlier bird species. So maybe there was a bird that wasn't quite a chicken but laid an egg that had a mutation making its offspring the first true chicken. In that case, the egg would come before the chicken because the egg's parent wasn't a chicken yet. But wait, then the egg inside the mother (or in the nest) would contain the chicken. So does the chicken exist when it's still an embryo? Or is it considered a chicken only after hatching?

Alternatively, if the first chicken had to come from a non-chicken ancestor, then that ancestor would have laid an egg containing the chicken. Therefore, the egg came first because the parent was not a chicken but the offspring was. But another angle: maybe the mutation happened in the parent's reproductive cells bef

>>>  Are there any other paradoxes related to the one I asked for you?


<think>

Okay, so the user just asked about other paradoxes similar to the "egg or chicken" question. Let me think about how to approach this.

First, I need to recall what a paradox is. Paradoxes are situations that seem contradictory or defy intuition but often reveal deeper truths when analyzed carefully. The egg/chicken problem is a causality dilemma—what comes first in a cyclical cause-effect relationship?

The user might be interested in similar thought experiments that challenge our understanding of time, logic, or definitions. Let me brainstorm some classic paradoxes.

Zeno's Paradoxes come to mind. They deal with motion and infinity, like the dichotomy paradox where you can't reach a destination because you have to cover half the distance infinitely. That's about infinite divisibility and motion, which is different but also a foundational question.

The Liar Paradox ("This statement is false") is another classic. It creates an unresolvable contradiction because if it's true, t

>>>  /bye


Terminating Ollama server...
Ollama server terminated gracefully.
